In [1]:
import os
import pickle
import glob
import gzip
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import collections

In [2]:
#2021
filename_mesh = 'mesh_list_2021.pickle'
with open(filename_mesh, 'rb') as f:
    mesh_list = pickle.load(f)

#メッシュから　メッシュIDと疾患/症状メッシュ、TreeNumber を取り出す
#疾患メッシュ：Cのうち C22（動物)とC23.888（症状）を除外したもの）＋F03（精神障害）
#症状メッシュ：C23.888
mesh_diseases = [descriptor for descriptor in tqdm(mesh_list) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
mesh_symptoms = [descriptor for descriptor in tqdm(mesh_list)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]
#さらにメッシュ名だけにする＋重複削除
mesh_diseases= pd.DataFrame(mesh_diseases)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()
mesh_symptoms= pd.DataFrame(mesh_symptoms)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()

len(mesh_list),len(mesh_diseases), len(mesh_symptoms)

<ipython-input-2-a73f21baf64c>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_diseases = [descriptor for descriptor in tqdm(mesh_list) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]


  0%|          | 0/29917 [00:00<?, ?it/s]

<ipython-input-2-a73f21baf64c>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_symptoms = [descriptor for descriptor in tqdm(mesh_list)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]


  0%|          | 0/29917 [00:00<?, ?it/s]

(29917, 4887, 394)

In [3]:
#2011
filename_mesh = 'mesh_list_2011.pickle'
with open(filename_mesh, 'rb') as f:
    mesh_list_2011 = pickle.load(f)

#メッシュから　メッシュIDと疾患/症状メッシュ、TreeNumber を取り出す
#疾患メッシュ：Cのうち C22（動物)とC23.888（症状）を除外したもの）＋F03（精神障害）
#症状メッシュ：C23.888
mesh_diseases_2011 = [descriptor for descriptor in tqdm(mesh_list_2011) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
mesh_symptoms_2011 = [descriptor for descriptor in tqdm(mesh_list_2011)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]
#さらにメッシュ名だけにする＋重複削除
mesh_diseases_2011= pd.DataFrame(mesh_diseases_2011)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()
mesh_symptoms_2011= pd.DataFrame(mesh_symptoms_2011)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()

len(mesh_list_2011),len(mesh_diseases_2011), len(mesh_symptoms_2011)

<ipython-input-3-14f0db5d5719>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_diseases_2011 = [descriptor for descriptor in tqdm(mesh_list_2011) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]


  0%|          | 0/26142 [00:00<?, ?it/s]

<ipython-input-3-14f0db5d5719>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_symptoms_2011 = [descriptor for descriptor in tqdm(mesh_list_2011)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]


  0%|          | 0/26142 [00:00<?, ?it/s]

(26142, 4475, 328)

In [4]:
#i[0] mesh unique id -> 2021, 2011どちらにもあるmesh_idの数:26019 
all_2021 =  [i[1] for i in mesh_list]
all_2011 =  [i[1] for i in mesh_list_2011]
all_common = list( set(all_2021) & set(all_2011))
print("2021MeSH:", len(all_2021))
print("2011MeSH:", len(all_2011))
print("2021, 2011どちらにもあるmesh_nameの数:", len(all_common))
only_2021 = sorted(list(set(all_2021) - set(all_common)))
only_2011 = sorted(list(set(all_2011) - set(all_common)))
print("2021にしかないタームの数:", len(only_2021))
print("2011にしかないタームの数:", len(only_2011))

2021MeSH: 29917
2011MeSH: 26142
2021, 2011どちらにもあるmesh_nameの数: 25546
2021にしかないタームの数: 4371
2011にしかないタームの数: 596


In [5]:
#disease
print("2021, 2011どちらにもあるdiseaseタームの数:", len(list(set(mesh_diseases) & set(mesh_diseases_2011))))
new_d = sorted(list(set(mesh_diseases) - set(mesh_diseases_2011)))
old_d = sorted(list(set(mesh_diseases_2011) - set(mesh_diseases)))
print("2021にしかないタームの数:", len(new_d))
print("2011にしかないタームの数:", len(old_d))

2021, 2011どちらにもあるdiseaseタームの数: 4409
2021にしかないタームの数: 478
2011にしかないタームの数: 66


In [6]:
#symptom
print("2021, 2011どちらにもあるsymptomタームの数:", len(list(set(mesh_symptoms) & set(mesh_symptoms_2011))))
new_s = sorted(list(set(mesh_symptoms) - set(mesh_symptoms_2011)))
old_s = sorted(list(set(mesh_symptoms_2011) - set(mesh_symptoms)))
print("2021にしかないタームの数:", len(new_s))
print("2011にしかないタームの数:", len(old_s))

2021, 2011どちらにもあるsymptomタームの数: 318
2021にしかないタームの数: 76
2011にしかないタームの数: 10


In [7]:
#疾患症状どちらも持つターム
ds_2021 = list(set(mesh_diseases) & set(mesh_symptoms))
ds_2011 = list(set(mesh_diseases_2011) & set(mesh_symptoms_2011))
print('2021_疾患症状どちらも持つターム数:',len(ds_2021))
print('2011_疾患症状どちらも持つターム数:',len(ds_2011))
print(ds_2021[:3])
ds = list(set(ds_2021) & set(ds_2011))
print("2021, 2011どちらにもある, 疾患症状どちらも持つタームの数:", len(ds))

2021_疾患症状どちらも持つターム数: 284
2011_疾患症状どちらも持つターム数: 250
['Hyperacusis', 'Ophthalmoplegic Migraine', 'Hearing Loss, Noise-Induced']
2021, 2011どちらにもある, 疾患症状どちらも持つタームの数: 231


In [8]:
new_ds = sorted(list(set(ds_2021) - set(ds)))
old_ds = sorted(list(set(ds_2011) - set(ds)))
print("2021にしかない,疾患症状どちらも持つタームの数:", len(new_ds))
print("2011にしかない,疾患症状どちらも持つタームの数:", len(old_ds))

2021にしかない,疾患症状どちらも持つタームの数: 53
2011にしかない,疾患症状どちらも持つタームの数: 19


In [9]:
only_d_2021 = sorted(list(set(mesh_diseases) - set(ds_2021)))
only_s_2021 = sorted(list(set(mesh_symptoms) - set(ds_2021)))

only_d_2011 = sorted(list(set(mesh_diseases_2011) - set(ds_2011)))
only_s_2011 = sorted(list(set(mesh_symptoms_2011) - set(ds_2011)))

print("2021_疾患のみターム数:", len(only_d_2021))
print("2021_症状のみターム数:", len(only_s_2021))
print('2021_疾患症状どちらも持つターム数:',len(ds_2021))
print("2011_疾患のみターム数:", len(only_d_2011))
print("2011_症状のみターム数:", len(only_s_2011))
print('2011_疾患症状どちらも持つターム数:',len(ds_2011))

2021_疾患のみターム数: 4603
2021_症状のみターム数: 110
2021_疾患症状どちらも持つターム数: 284
2011_疾患のみターム数: 4225
2011_症状のみターム数: 78
2011_疾患症状どちらも持つターム数: 250


In [10]:
only_d_common = list(set(only_d_2021) & set(only_d_2011))
only_s_common = list(set(only_s_2021) & set(only_s_2011))
print("2021, 2011どちらにもある疾患のみターム:",len(only_d_common))
print("2021, 2011どちらにもある症状のみターム:",len(only_s_common))

2021, 2011どちらにもある疾患のみターム: 4161
2021, 2011どちらにもある症状のみターム: 76


In [11]:
only_d_only_2021 = sorted(list(set(only_d_2021) - set(only_d_common)))
only_d_only_2011 = sorted(list(set(only_d_2011) - set(only_d_common)))
print("2021にしかない疾患のみターム:", len(only_d_only_2021))
print("2011にしかない疾患のみターム:", len(only_d_only_2011))
#print("2021, 2011どちらにもある疾患のみターム", len(only_d_common))

2021にしかない疾患のみターム: 442
2011にしかない疾患のみターム: 64


In [12]:
only_s_only_2021 = sorted(list(set(only_s_2021) - set(only_s_common)))
only_s_only_2011 = sorted(list(set(only_s_2011) - set(only_s_common)))
print("2021にしかない症状のみターム:", len(only_s_only_2021))
print("2011にしかない症状のみターム:", len(only_s_only_2011))
#print("2021, 2011どちらにもある症状のみターム", len(only_s_common))

2021にしかない症状のみターム: 34
2011にしかない症状のみターム: 2


In [13]:
only_s_only_2011

['Anoxia', 'Colic']

In [14]:
only_s_only_2021

['Acute Pain',
 'Alert Fatigue, Health Personnel',
 'Back Pain',
 'Body-Weight Trajectory',
 'Breakthrough Pain',
 'Cancer Pain',
 'Cellulite',
 'Chronic Pain',
 'Compassion Fatigue',
 'Dizziness',
 'Facial Pain',
 'Failure to Thrive',
 'Food Intolerance',
 'Gestational Weight Gain',
 'Headache',
 'Hypoxia',
 'Labor Pain',
 'Low Back Pain',
 'Lower Urinary Tract Symptoms',
 'Mastodynia',
 'Medically Unexplained Symptoms',
 'Neck Pain',
 'Nociceptive Pain',
 'Pain',
 'Pain, Intractable',
 'Pain, Procedural',
 'Pain, Referred',
 'Pelvic Girdle Pain',
 'Prodromal Symptoms',
 'Rhinorrhea',
 'Sleepiness',
 'Torticollis',
 'Travel-Related Illness',
 'Visceral Pain']